## Notebook 1: Data Ingestion, Standardization, and Cleaning

In this notebook, we will build the foundation for our Australian Open 2026 prediction model. Our raw data consists of individual CSV files for every year of professional tennis match results (from 1968 to 2026).

Our goals are:
1. Ingest: Load match data sequentially from 1968 through the current 2026 season.
1. Concatenate: Merge these yearly files into a single, unified "Master DataFrame."
1. Clean: Handle missing values logically. We will specific tennis domain knowledge to fill gaps (e.g., Unseeded players are not "Seed 0", they are "Seed 100+").
1. Standardize: Ensure date columns are actual datetime objects and categorical data is consistent.
1. Export: Save the cleaned data as a single CSV file (master_data_cleaned.csv) for use in feature engineering.

## 1. Import Libraries

We will rely on pandas for data manipulation and numpy for numerical operations.

In [251]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Set pandas display options to ensure we can see all columns during inspection
pd.set_option('display.max_columns', None)

## 2.. Data Ingestion

Instead of loading everything blindly, we will iterate through the years 1968 to 2026. This allows us to explicitly control the scope of our historical data.

Our data is stored under the "./data/" directory with each file named "YYYY.csv".

In [252]:
# Define the path and the range of years we want to analyze
data_path = './data/'
start_year = 1968
end_year = 2026  # Includes the current year's data up to the AO

# Initialize an empty list to store each year's dataframe
all_matches = []

print(f"Starting data ingestion from {start_year} to {end_year}...")

# Loop through each year
for year in range(start_year, end_year + 1):
    file_path = f"{data_path}{year}.csv"
    
    # Check if the file exists before trying to read it
    if os.path.exists(file_path):
        try:
            # Read the CSV file
            # low_memory=False helps pandas guess data types accurately for large files
            current_df = pd.read_csv(file_path, low_memory=False)
            
            # Append to our list
            all_matches.append(current_df)
            
            # Print status every 10 years to avoid cluttering the output
            if year % 10 == 0 or year == end_year:
                print(f"Successfully loaded: {year}.csv ({len(current_df)} matches)")
                
        except Exception as e:
            print(f"Error reading {year}.csv: {e}")
    else:
        print(f"Warning: File not found for year {year}")

# Concatenate all yearly dataframes into one Master DataFrame
df_master = pd.concat(all_matches, ignore_index=True)

print("-" * 50)
print(f"Ingestion complete.\nTotal matches loaded: {len(df_master):,}")

Starting data ingestion from 1968 to 2026...
Successfully loaded: 1970.csv (3236 matches)
Successfully loaded: 1980.csv (3909 matches)
Successfully loaded: 1990.csv (3683 matches)
Successfully loaded: 2000.csv (3378 matches)
Successfully loaded: 2010.csv (3030 matches)
Successfully loaded: 2020.csv (1466 matches)
Successfully loaded: 2026.csv (137 matches)
--------------------------------------------------
Ingestion complete.
Total matches loaded: 198,063


## 3. Date Parsing and Sorting

Raw CSV data usually stores dates as integers (e.g., 20230115) or strings. For time-series forecasting, we must convert these to proper Python datetime objects. We also sort the data chronologically to ensure our future model doesn't "peek" at future matches during training.

In [253]:
# Convert 'tourney_date' to datetime objects
# errors='coerce' will turn any corrupted dates into NaT (Not a Time) so they don't crash the script
df_master['tourney_date'] = pd.to_datetime(df_master['tourney_date'], format='%Y%m%d', errors='coerce')

# Sort the data:
# 1. By Date (Primary)
# 2. By Tournament ID (Secondary - keeps tournament matches together)
# 3. By Match Number (Tertiary - orders R128 -> R64 -> Final)
df_master = df_master.sort_values(by=['tourney_date', 'tourney_id', 'match_num']).reset_index(drop=True)

# Verify the range
print(f"Date Range: {df_master['tourney_date'].min()} to {df_master['tourney_date'].max()}")

Date Range: 1968-01-01 00:00:00 to 2026-01-17 00:00:00


## 4. Handling Missing Values (Data Imputation)
Tennis data often has missing values (NaN). How we fill them matters significantly for model performance.

### 4.1. Rankings
Logic: In tennis, a Rank of 1 is the best. If we fill missing ranks with 0, the model will think that player is better than the World No. 1.

Fix: We replace missing ranks with 9999 to signify "Unranked" or "Low Rank".

In [254]:
# Fill missing rankings with 9999
df_master['winner_rank'] = df_master['winner_rank'].fillna(9999).astype(int)
df_master['loser_rank'] = df_master['loser_rank'].fillna(9999).astype(int)

# Fill missing ranking points with 0 (Unranked players have 0 points)
df_master['winner_rank_points'] = df_master['winner_rank_points'].fillna(0).astype(int)
df_master['loser_rank_points'] = df_master['loser_rank_points'].fillna(0).astype(int)

### 4.2. Seeds & entries
Logic for seeds: An unseeded player is effectively a very low seed. We use 100 to represent this. 

Logic for entries: Missing entry type usually means "Standard" (Direct Acceptance).

In [255]:
# Fill missing seeds with 100
df_master['winner_seed'] = df_master['winner_seed'].fillna(100).astype(int)
df_master['loser_seed'] = df_master['loser_seed'].fillna(100).astype(int)

# Fill missing entries with 'Std'
df_master['winner_entry'] = df_master['winner_entry'].fillna('Std')
df_master['loser_entry'] = df_master['loser_entry'].fillna('Std')

### 4.3. Physical attributes (height & age)
Logic: A height of 0 cm or an age of 0 is impossible. 

Fix: We fill missing heights and ages with the median of the dataset. This is robust to outliers.

In [256]:
# Calculate medians based on the entire dataset
# We use nanmedian to ignore existing NaNs during calculation
avg_winner_ht = df_master['winner_ht'].median()
avg_loser_ht = df_master['loser_ht'].median()
avg_winner_age = df_master['winner_age'].median()
avg_loser_age = df_master['loser_age'].median()

# Apply imputation
df_master['winner_ht'] = df_master['winner_ht'].fillna(avg_winner_ht)
df_master['loser_ht'] = df_master['loser_ht'].fillna(avg_loser_ht)
df_master['winner_age'] = df_master['winner_age'].fillna(avg_winner_age)
df_master['loser_age'] = df_master['loser_age'].fillna(avg_loser_age)

### 4.4. Match statistics
Logic: If match stats (z.B., aces, double faults, break points) are missing, it usually means the match statistics weren't recorded for that tournament, which are common in Futures/Challengers level.

Fix: We assume 0 for these values.

In [257]:
# List of technical match statistics columns
stat_cols = [
    'minutes', 
    'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced',
    'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced'
]

# Fill missing stats with 0
for col in stat_cols:
    if col in df_master.columns:
        df_master[col] = df_master[col].fillna(0).astype(float)

### 4.5. Draw Size
Logic: We cannot just use the median (32) for everything because a Grand Slam (128) is much harder to win than a Challenger (32). 

Fix: We map the draw_size based on the tourney_level.

In [258]:
# Map draw size to Tournament Level
draw_size_map = {
    'G': 128,  # Grand Slam
    'M': 64,   # Masters 1000 (standardized to 64, though some are 56/96)
    'A': 32,   # ATP Tour (general)
    '250': 32, # Specific level label
    '500': 32, # Specific level label
    'C': 32,   # Challengers
    'S': 32,   # Satellites/Futures
    'U': 18,   # United Cup
    'F': 8,    # ATP Finals (year-end)
    'D': 4,    # Davis Cup
}

# Fill based on the map
df_master['draw_size'] = df_master['draw_size'].fillna(df_master['tourney_level'].map(draw_size_map))

# Fallback: If level was unknown, use 32 (standard ATP size)
df_master['draw_size'] = df_master['draw_size'].fillna(32).astype(int)

# Fix missing match numbers (default to 1)
df_master['match_num'] = df_master['match_num'].fillna(1).astype(int)

### 4.6. Other categorical Data
Logic: We cannot mathematically impute a player's country or dominant hand or whether the court is indoor vs outdoor.

Fix: We create a new category called "Unknown". This allows the random forest to see "Unknown" as a distinct group rather than crashing.

In [259]:
categorical_cols = ['surface', 'indoor', 'winner_hand', 'loser_hand', 'winner_ioc', 'loser_ioc']

for col in categorical_cols:
    df_master[col] = df_master[col].fillna('Unknown')

# Standardize hand: Sometimes data has 'Right', 'R', or 'nan'. 
# We map them to 'R', 'L', and 'U', respectively.
normalization_map = {'Right': 'R', 'Left': 'L', 'nan': 'U', 'Unknown': 'U'}
df_master['winner_hand'] = df_master['winner_hand'].replace(normalization_map)
df_master['loser_hand'] = df_master['loser_hand'].replace(normalization_map)

## 5. Final Cleanup and Export
We drop the small number of rows where the score or loser_id is missing. These represent corrupted match records (e.g., walkovers without data) that cannot be used for training.

Finally, we verify that our data is clean and save it as CSV.

In [260]:
# Check if any nulls remain
print("\nRemaining Missing Values (Top 10 columns):")
print(df_master.isnull().sum().sort_values(ascending=False).head(5))

# Drop corrupted rows (score or player IDs missing)
df_master = df_master.dropna(subset=['score', 'winner_id', 'loser_id'])

# Finally verify if any nulls remain
missing_sum = df_master.isnull().sum().sum()
print(f"\nRemaining Missing Values: {missing_sum}")

# Define output filename
output_file = 'master_data_cleaned.csv'

# Save the clean file
# Set index=False to avoid creating an unnamed index column
# When we load this CSV in the next notebook, we need to re-parse the date column because CSVs store dates as strings.
df_master.to_csv(output_file, index=False)
print(f"\nSUCCESS: Clean dataset saved to {output_file}")


Remaining Missing Values (Top 10 columns):
score         7
loser_id      1
tourney_id    0
w_2ndWon      0
best_of       0
dtype: int64

Remaining Missing Values: 0

SUCCESS: Clean dataset saved to master_data_cleaned.csv
